# Retrieve data from our main submission result (complete error mitigation with 96% fidelity) and use parts of calibration data from that to do CTMP and tensored error mitigation

## Imports

Begin by importing the necessary packages and defining the functions we will need for the stabilizer measurements.

In [1]:
### install Qiskit and other modules if you don't have them already
#!pip install -r dependencies/requirements.txt --quiet

In [2]:
# Qiskit module
from qiskit import QuantumCircuit
import qiskit.circuit.library as circuit_library
import qiskit.quantum_info as qi
import qiskit.ignis.mitigation as mit
import qiskit.compiler.transpile as transpile

### Added from SWAP challenge
from qiskit import transpile, schedule, assemble, execute

# Qiskit tools for noisy simulation
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.utils import insert_noise

# Qiskit tools for running and monitoring jobs
from qiskit import execute
from qiskit.tools.monitor import job_monitor

# Other imports
import numpy as np

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load IBMQ Account data
from qiskit import IBMQ
IBMQ.save_account('2b8218f150600ba6d35be3868c1b178cce4f5139ac2d56ea06d87616b9b84fca017047b84fe28a3ceb7629ec56f36490a4e56375c081b4de68adeab5fb146c25', overwrite = 'True')
IBMQ.load_account()

# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science')
backend = provider.get_backend('ibmq_casablanca')
properties = backend.properties()

## Preparing graph states (just for definition of stabilizers, results are all retrieved)

In [4]:
num_qubits = 7

# adjacency matrix for `ibmq_casablanca`
adjmat = [
    [0, 1, 0, 0, 0, 0, 0], 
    [1, 0, 1, 1, 0, 0, 0], 
    [0, 1, 0, 0, 0, 0, 0], 
    [0, 1, 0, 0, 0, 1, 0], 
    [0, 0, 0, 0, 0, 1, 0], 
    [0, 0, 0, 1, 1, 0, 1], 
    [0, 0, 0, 0, 0, 1, 0]]

In [5]:
state_circuit = QuantumCircuit(7)
state_circuit.h(0)
state_circuit.h(2)
state_circuit.h(3)
state_circuit.h(4)
state_circuit.h(6)
state_circuit.cx(0, 1)
state_circuit.cx(3, 5)
state_circuit.cx(2, 1)
state_circuit.cx(4, 5)
state_circuit.cx(3, 1)
state_circuit.cx(6, 5)
state_circuit.h(1)
state_circuit.h(5)
#state_circuit = transpile(state_circuit)
state_circuit.draw()




┌───┐                    
q_0: ┤ H ├──■─────────────────
     └───┘┌─┴─┐┌───┐┌───┐┌───┐
q_1: ─────┤ X ├┤ X ├┤ X ├┤ H ├
     ┌───┐└───┘└─┬─┘└─┬─┘└───┘
q_2: ┤ H ├───────■────┼───────
     ├───┤            │       
q_3: ┤ H ├──■─────────■───────
     ├───┤  │                 
q_4: ┤ H ├──┼────■────────────
     └───┘┌─┴─┐┌─┴─┐┌───┐┌───┐
q_5: ─────┤ X ├┤ X ├┤ X ├┤ H ├
     ┌───┐└───┘└───┘└─┬─┘└───┘
q_6: ┤ H ├────────────■───────
     └───┘

## Creating and measuring stabilizers

In [6]:
def compute_stabilizer_group(circuit):
    """Compute the stabilizer group for stabilizer circuit."""
    state = qi.Statevector.from_instruction(circuit)
    labels = []
    for i in qi.pauli_basis(state.num_qubits):
        val = round(qi.state_fidelity(i.to_matrix()[0], state, validate=False))
        if val != 0:
            label = i.to_labels()[0]
            if val == 1:
                label = '+' + label
            else:
                label = '-' + label
            labels.append(label)
    return labels

def stabilizer_coeff_pauli(stabilizer):
    """Return the 1 or -1 coeff and Pauli label."""
    coeff = 1
    pauli = coeff
    if stabilizer[0] == '-':
        coeff = -1
    if stabilizer[0] in ['+', '-']:
        pauli = stabilizer[1:]
    else:
        pauli = stabilizer
    return coeff, pauli

def stabilizer_measure_circuit(stabilizer, initial_circuit=None):
    """Return a stabilizer measurement circuits.
    
    Args:
        stabilizer (str): a stabilizer string
        initial_circuit (QuantumCircuit): Optional, the initial circuit.
    
    Returns:
        QuantumCircuit: the circuit with stabilizer measurements.
    """
    _, pauli = stabilizer_coeff_pauli(stabilizer)
    if initial_circuit is None:
        circ = QuantumCircuit(len(pauli))
    else:
        circ = initial_circuit.copy()
    for i, s in enumerate(reversed(pauli)):
        if s == 'X':
            circ.h(i)
        if s == 'Y':
            circ.sdg(i)
            circ.h(i)
    circ.measure_all()
    return circ

In [7]:
## Compute the stabilizers for this graph state
generators = qi.Clifford(state_circuit).stabilizer.pauli.to_labels()
stabilizers = compute_stabilizer_group(state_circuit)
print('Stabilizers:', stabilizers)
print('Generators:', generators)

Stabilizers: ['+IIIIIII', '+IIIIIZX', '+IIIIXIX', '+IIIIXZI', '-IIIZYXY', '+IIIZYYZ', '+IIIZZXZ', '+IIIZZYY', '+IIXXIIX', '+IIXXIZI', '+IIXXXII', '+IIXXXZX', '-IIXYYXZ', '-IIXYYYY', '-IIXYZXY', '+IIXYZYZ', '+IZIXIIX', '+IZIXIZI', '+IZIXXII', '+IZIXXZX', '-IZIYYXZ', '-IZIYYYY', '-IZIYZXY', '+IZIYZYZ', '+IZXIIII', '+IZXIIZX', '+IZXIXIX', '+IZXIXZI', '-IZXZYXY', '+IZXZYYZ', '+IZXZZXZ', '+IZXZZYY', '+XIIXIIX', '+XIIXIZI', '+XIIXXII', '+XIIXXZX', '-XIIYYXZ', '-XIIYYYY', '-XIIYZXY', '+XIIYZYZ', '+XIXIIII', '+XIXIIZX', '+XIXIXIX', '+XIXIXZI', '-XIXZYXY', '+XIXZYYZ', '+XIXZZXZ', '+XIXZZYY', '+XZIIIII', '+XZIIIZX', '+XZIIXIX', '+XZIIXZI', '-XZIZYXY', '+XZIZYYZ', '+XZIZZXZ', '+XZIZZYY', '+XZXXIIX', '+XZXXIZI', '+XZXXXII', '+XZXXXZX', '-XZXYYXZ', '-XZXYYYY', '-XZXYZXY', '+XZXYZYZ', '+YXYIYXY', '-YXYIYYZ', '-YXYIZXZ', '-YXYIZYY', '-YXYZIII', '-YXYZIZX', '-YXYZXIX', '-YXYZXZI', '-YXZXYXZ', '-YXZXYYY', '-YXZXZXY', '+YXZXZYZ', '-YXZYIIX', '-YXZYIZI', '-YXZYXII', '-YXZYXZX', '-YYYXYXZ', '-YYYXYYY', '-

## Here we first retrieve results linked to CTMP error mitigation method

In [8]:
labels = ['0000000', '0000011', '0000101', '0000110', 
          '0001001', '0001010', '0001100', 
          '0010001', '0010010', '0010100', '0011000', 
          '0100001', '0100010', '0100100', '0101000', '0110000', 
          '1000001', '1000010', '1000100', '1001000', '1010000', '1100000', 
          '1111111']
meas_cal_circuits, metadata = mit.expval_meas_mitigator_circuits(num_qubits, labels = labels)

In [9]:
[meas_cal_circuits_full, state_labels] = mit.complete_meas_cal(range(num_qubits))

## Here running on the QASM simulator is just to get a copy of the data structure that will be passed to the functions for data fitting. The actual content of these retrieved jobs will be over-written by the retrieved data (complete mitigation, 96%).

In [10]:
backend = QasmSimulator.from_backend(provider.get_backend('ibmq_casablanca'))

In [11]:
reps = 16

In [12]:

all_jobs_mit_1 = []

for ii in range(reps):

    # Run QPT on backend
    shots = 8192
    il = [0,1,2,3,4,5,6]
    job_mit_backend = execute(meas_cal_circuits, backend, shots=shots, initial_layout=il)
    all_jobs_mit_1.append(job_mit_backend)
    

In [13]:
for job in all_jobs_mit_1:
    job_monitor(job)
    try:
        if job.error_message() is not None:
            print(job.error_message())
    except:
        pass

Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run


In [14]:
result_mit_backend_1 = []


    
for job in all_jobs_mit_1:
    result_mit_backend_1.append(job.result())

In [15]:
metadata

[{'experiment': 'meas_mit', 'cal': '0000000', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0000011', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0000101', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0000110', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0001001', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0001010', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0001100', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0010001', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0010010', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0010100', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0011000', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0100001', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0100010', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0100100', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0101000', 'method': 'CTMP'},
 {'experiment': 'meas_mit

## Post-processing and computing fidelities

Retrieve data from our main results.

In [16]:
# retrieve jobs afterwards



backend = provider.get_backend('ibmq_casablanca')


msg = """
Job IDs (1/16): 
 measurement calibration: 603a61a3ead6ee2ea6edcba4
 stabilizer measurements: 603a61a16ebbe16d1371f0f5
Job IDs (2/16): 
 measurement calibration: 603a61a9a62407694fc8a419
 stabilizer measurements: 603a61a75728cef54b2be715
Job IDs (3/16): 
 measurement calibration: 603a61af6760ce1a6560cc5a
 stabilizer measurements: 603a61ad8c2165fff242d2ac
Job IDs (4/16): 
 measurement calibration: 603a61b55728ce30842be717
 stabilizer measurements: 603a61b4fcca6b33494c6082
Job IDs (5/16): 
 measurement calibration: 603a61bbc6d1d179d3f998de
 stabilizer measurements: 603a61b96ebbe1338171f0f8
Job IDs (6/16): 
 measurement calibration: 603a61c1fcca6b86e54c6084
 stabilizer measurements: 603a61bfa624073dfdc8a41c
Job IDs (7/16): 
 measurement calibration: 603a61c7c6d1d17877f998e0
 stabilizer measurements: 603a61c56760cef9e660cc5e
Job IDs (8/16): 
 measurement calibration: 603a61ccc6d1d1cb36f998e1
 stabilizer measurements: 603a61cbe8504d2d1e3eba73
Job IDs (9/16): 
 measurement calibration: 603a61d28c21656a7d42d2ae
 stabilizer measurements: 603a61d1a624075b98c8a41f
Job IDs (10/16): 
 measurement calibration: 603a61d8a6240798e0c8a421
 stabilizer measurements: 603a61d65728ce804d2be719
Job IDs (11/16): 
 measurement calibration: 603a61e0a624075e22c8a423
 stabilizer measurements: 603a61de742e5347d4801b44
Job IDs (12/16): 
 measurement calibration: 603a61e76760ce92a860cc61
 stabilizer measurements: 603a61e68c21657f9d42d2af
Job IDs (13/16): 
 measurement calibration: 603a61edead6ee6070edcba8
 stabilizer measurements: 603a61ebfcca6b62e64c6088
Job IDs (14/16): 
 measurement calibration: 603a61f2c6d1d18df1f998e4
 stabilizer measurements: 603a61f16760ce0a7a60cc62
Job IDs (15/16): 
 measurement calibration: 603a61f8c6d1d18051f998e6
 stabilizer measurements: 603a61f68c2165055d42d2b1
Job IDs (16/16): 
 measurement calibration: 603a61fe5728ce78972be71d
 stabilizer measurements: 603a61fc6760cece1960cc65
"""

all_jobs = []
all_jobs_mit = []

for i in msg.split('\n'):
    if i[:26] == " measurement calibration: ":
        all_jobs_mit.append(backend.retrieve_job(i[26:]))
    elif i[:26] == " stabilizer measurements: ":
        all_jobs.append(backend.retrieve_job(i[26:]))


print(all_jobs)
print(all_jobs_mit)

[<qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa805b8c70>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa62271cd0>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6226af40>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6157a5e0>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6157a580>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa80980490>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa62275310>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6227c100>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6227c9d0>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6227ceb0>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6227c550>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6227cb80>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6225f550>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6225f760>, <qiskit.providers.i

In [17]:
result_backend = []
result_mit_backend = []



for job in all_jobs:
    # Retrieve results (this may take a while depending on the queue)
    result_backend.append(job.result())
    
for job in all_jobs_mit:
    result_mit_backend.append(job.result())

### Transfer the calibration data from the complete mitigation results to the appropriate CTMP fitting jobs

In [18]:

result_mit_backend_1_copy = result_mit_backend_1
for i in range(reps): 
    
    result_mit_backend_1_copy[i].results[0].data = result_mit_backend[i].results[0].data
    result_mit_backend_1_copy[i].results[1].data = result_mit_backend[i].results[3].data
    result_mit_backend_1_copy[i].results[2].data = result_mit_backend[i].results[5].data
    result_mit_backend_1_copy[i].results[3].data = result_mit_backend[i].results[6].data
    result_mit_backend_1_copy[i].results[4].data = result_mit_backend[i].results[9].data
    result_mit_backend_1_copy[i].results[5].data = result_mit_backend[i].results[10].data
    result_mit_backend_1_copy[i].results[6].data = result_mit_backend[i].results[12].data
    result_mit_backend_1_copy[i].results[7].data = result_mit_backend[i].results[17].data
    result_mit_backend_1_copy[i].results[8].data = result_mit_backend[i].results[18].data
    result_mit_backend_1_copy[i].results[9].data = result_mit_backend[i].results[20].data
    result_mit_backend_1_copy[i].results[10].data = result_mit_backend[i].results[24].data
    result_mit_backend_1_copy[i].results[11].data = result_mit_backend[i].results[33].data
    result_mit_backend_1_copy[i].results[12].data = result_mit_backend[i].results[34].data
    result_mit_backend_1_copy[i].results[13].data = result_mit_backend[i].results[36].data
    result_mit_backend_1_copy[i].results[14].data = result_mit_backend[i].results[40].data
    result_mit_backend_1_copy[i].results[15].data = result_mit_backend[i].results[48].data
    result_mit_backend_1_copy[i].results[16].data = result_mit_backend[i].results[65].data
    result_mit_backend_1_copy[i].results[17].data = result_mit_backend[i].results[66].data
    result_mit_backend_1_copy[i].results[18].data = result_mit_backend[i].results[68].data
    result_mit_backend_1_copy[i].results[19].data = result_mit_backend[i].results[72].data
    result_mit_backend_1_copy[i].results[20].data = result_mit_backend[i].results[80].data
    result_mit_backend_1_copy[i].results[21].data = result_mit_backend[i].results[96].data
    result_mit_backend_1_copy[i].results[22].data = result_mit_backend[i].results[127].data



### Now re-compute fidelities using the CTMP mitigation method

In [19]:
def stabilizer_measure_diagonal(stabilizer):
    """Return the diagonal vector for a stabilizer measurement.
    
    Args:
        stabilizer (str): a stabilizer string
    
    Returns:
        np.ndarray: the diagonal for measurement in the stabilizer basis.
    """
    coeff, pauli = stabilizer_coeff_pauli(stabilizer)
    diag = np.array([1])
    for s in reversed(pauli):
        if s == 'I':
            tmp = np.array([1, 1])
        else:
            tmp = np.array([1, -1])
        diag = np.kron(tmp, diag)
    return coeff * diag
    
def stabilizer_fidelity(expvals, stddevs=None):
    """Compute stabilizer state fidelity from stabilizer expvals."""
    mean = np.mean(expvals)
    if stddevs is None:
        return mean
    stddev = np.sqrt(np.sum(stddevs ** 2))
    return mean, stddev

In [20]:
def stabilizer_expvals(result, stabilizers, meas_mitigator=None):
    """Compute expectation values from stabilizer measurement results."""

    ### YOUR CODE GOES HERE -- START
    
    expvals = []
    stddevs = []
    for i, stab in enumerate(stabilizers):
        expval, stddev = mit.expectation_value(
            result.get_counts(i),
            diagonal=stabilizer_measure_diagonal(stab),
            meas_mitigator=meas_mitigator)
        expvals.append(expval)
        stddevs.append(stddev)
    return np.array(expvals), np.array(stddevs)
    ### YOUR CODE GOES HERE -- END

In [21]:
## Mitigate the stabilizer expectation values 
F_nomit_backend = []
F_mit_backend = []


for ii in range(reps):
    # Unmitigated Expectation Values
    expvals_nomit_b, stddevs_nomit_b = stabilizer_expvals(
        result_backend[ii], stabilizers)
  
    # Fit measurement error mitigators
    mitigator_backend = mit.ExpvalMeasMitigatorFitter(result_mit_backend_1_copy[ii], metadata).fit()

    # Measurement error mitigated expectation values
    expvals_mit_b, stddevs_mit_b = stabilizer_expvals(
        result_backend[ii], stabilizers, meas_mitigator=mitigator_backend)
    
    # save the fidelities for this iteration
    F_nomit_backend.append(stabilizer_fidelity(expvals_nomit_b, stddevs_nomit_b)[0])
    F_mit_backend.append(stabilizer_fidelity(expvals_mit_b, stddevs_mit_b)[0])

## Report the CTMP fidelity estimate: 95.2 +- 1.7%, very similar to complete's

In [22]:
## The final results using CTMP mitigation

print('Graph-state fidelity estimates')
print('\nNo mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, np.mean(F_nomit_backend), np.std(F_nomit_backend)))

print('\nCTMP error mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, np.mean(F_mit_backend), np.std(F_mit_backend)))

Graph-state fidelity estimates

No mitigation
F(ibmq_casablanca) = 0.729 ± 0.011

CTMP error mitigation
F(ibmq_casablanca) = 0.952 ± 0.017


# Hereafter we shall recompute the results using tensored error mitigation.

In [23]:
meas_cal_circuits, metadata = mit.expval_meas_mitigator_circuits(num_qubits, method = 'tensored')
backend = QasmSimulator.from_backend(provider.get_backend('ibmq_casablanca'))

In [24]:
all_jobs_mit_2 = []

for ii in range(reps):

    # Run QPT on backend
    shots = 8192
    il = [0,1,2,3,4,5,6]
    job_mit_backend = execute(meas_cal_circuits, backend, shots=shots, initial_layout=il)
    all_jobs_mit_2.append(job_mit_backend)

In [25]:
for job in all_jobs_mit_2:
    job_monitor(job)
    try:
        if job.error_message() is not None:
            print(job.error_message())
    except:
        pass

Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run


In [26]:
result_mit_backend_2 = []    
for job in all_jobs_mit_1:
    result_mit_backend_2.append(job.result())

In [27]:
# retrieve jobs afterwards



backend = provider.get_backend('ibmq_casablanca')


msg = """
Job IDs (1/16): 
 measurement calibration: 603a61a3ead6ee2ea6edcba4
 stabilizer measurements: 603a61a16ebbe16d1371f0f5
Job IDs (2/16): 
 measurement calibration: 603a61a9a62407694fc8a419
 stabilizer measurements: 603a61a75728cef54b2be715
Job IDs (3/16): 
 measurement calibration: 603a61af6760ce1a6560cc5a
 stabilizer measurements: 603a61ad8c2165fff242d2ac
Job IDs (4/16): 
 measurement calibration: 603a61b55728ce30842be717
 stabilizer measurements: 603a61b4fcca6b33494c6082
Job IDs (5/16): 
 measurement calibration: 603a61bbc6d1d179d3f998de
 stabilizer measurements: 603a61b96ebbe1338171f0f8
Job IDs (6/16): 
 measurement calibration: 603a61c1fcca6b86e54c6084
 stabilizer measurements: 603a61bfa624073dfdc8a41c
Job IDs (7/16): 
 measurement calibration: 603a61c7c6d1d17877f998e0
 stabilizer measurements: 603a61c56760cef9e660cc5e
Job IDs (8/16): 
 measurement calibration: 603a61ccc6d1d1cb36f998e1
 stabilizer measurements: 603a61cbe8504d2d1e3eba73
Job IDs (9/16): 
 measurement calibration: 603a61d28c21656a7d42d2ae
 stabilizer measurements: 603a61d1a624075b98c8a41f
Job IDs (10/16): 
 measurement calibration: 603a61d8a6240798e0c8a421
 stabilizer measurements: 603a61d65728ce804d2be719
Job IDs (11/16): 
 measurement calibration: 603a61e0a624075e22c8a423
 stabilizer measurements: 603a61de742e5347d4801b44
Job IDs (12/16): 
 measurement calibration: 603a61e76760ce92a860cc61
 stabilizer measurements: 603a61e68c21657f9d42d2af
Job IDs (13/16): 
 measurement calibration: 603a61edead6ee6070edcba8
 stabilizer measurements: 603a61ebfcca6b62e64c6088
Job IDs (14/16): 
 measurement calibration: 603a61f2c6d1d18df1f998e4
 stabilizer measurements: 603a61f16760ce0a7a60cc62
Job IDs (15/16): 
 measurement calibration: 603a61f8c6d1d18051f998e6
 stabilizer measurements: 603a61f68c2165055d42d2b1
Job IDs (16/16): 
 measurement calibration: 603a61fe5728ce78972be71d
 stabilizer measurements: 603a61fc6760cece1960cc65
"""

all_jobs = []
all_jobs_mit = []

for i in msg.split('\n'):
    if i[:26] == " measurement calibration: ":
        all_jobs_mit.append(backend.retrieve_job(i[26:]))
    elif i[:26] == " stabilizer measurements: ":
        all_jobs.append(backend.retrieve_job(i[26:]))


print(all_jobs)
print(all_jobs_mit)

[<qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6226a640>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa804f5e80>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa80980490>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa62275a30>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6157a970>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa62271cd0>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6528f1f0>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6528f550>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6528f160>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6227ceb0>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6227c0a0>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6227c100>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa65289e20>, <qiskit.providers.ibmq.job.ibmqjob.IBMQJob object at 0x7ffa6227c160>, <qiskit.providers.i

In [28]:
result_backend = []
result_mit_backend = []



for job in all_jobs:
    # Retrieve results (this may take a while depending on the queue)
    result_backend.append(job.result())
    
for job in all_jobs_mit:
    result_mit_backend.append(job.result())

In [29]:
result_mit_backend_2_copy = result_mit_backend_2
for i in range(reps): 
    
    result_mit_backend_2_copy[i].results[0].data = result_mit_backend[i].results[0].data
    result_mit_backend_2_copy[i].results[1].data = result_mit_backend[i].results[-1].data

In [30]:
def stabilizer_measure_diagonal(stabilizer):
    """Return the diagonal vector for a stabilizer measurement.
    
    Args:
        stabilizer (str): a stabilizer string
    
    Returns:
        np.ndarray: the diagonal for measurement in the stabilizer basis.
    """
    coeff, pauli = stabilizer_coeff_pauli(stabilizer)
    diag = np.array([1])
    for s in reversed(pauli):
        if s == 'I':
            tmp = np.array([1, 1])
        else:
            tmp = np.array([1, -1])
        diag = np.kron(tmp, diag)
    return coeff * diag
    
def stabilizer_fidelity(expvals, stddevs=None):
    """Compute stabilizer state fidelity from stabilizer expvals."""
    mean = np.mean(expvals)
    if stddevs is None:
        return mean
    stddev = np.sqrt(np.sum(stddevs ** 2))
    return mean, stddev

In [31]:
def stabilizer_expvals(result, stabilizers, meas_mitigator=None):
    """Compute expectation values from stabilizer measurement results."""

    ### YOUR CODE GOES HERE -- START
    
    expvals = []
    stddevs = []
    for i, stab in enumerate(stabilizers):
        expval, stddev = mit.expectation_value(
            result.get_counts(i),
            diagonal=stabilizer_measure_diagonal(stab),
            meas_mitigator=meas_mitigator)
        expvals.append(expval)
        stddevs.append(stddev)
    return np.array(expvals), np.array(stddevs)
    ### YOUR CODE GOES HERE -- END

In [32]:
## Mitigate the stabilizer expectation values 
F_nomit_backend = []
F_mit_backend = []


for ii in range(reps):
    # Unmitigated Expectation Values
    expvals_nomit_b, stddevs_nomit_b = stabilizer_expvals(
        result_backend[ii], stabilizers)
  
    # Fit measurement error mitigators
    mitigator_backend = mit.ExpvalMeasMitigatorFitter(result_mit_backend_2_copy[ii], metadata).fit()

    # Measurement error mitigated expectation values
    expvals_mit_b, stddevs_mit_b = stabilizer_expvals(
        result_backend[ii], stabilizers, meas_mitigator=mitigator_backend)
    
    # save the fidelities for this iteration
    F_nomit_backend.append(stabilizer_fidelity(expvals_nomit_b, stddevs_nomit_b)[0])
    F_mit_backend.append(stabilizer_fidelity(expvals_mit_b, stddevs_mit_b)[0])

## Report the tensored fidelity estimate: 105 +- 3%, significantly over-estimation compared to complete's

In [33]:
## The final results from tensored mitigation

print('Graph-state fidelity estimates')
print('\nNo mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, np.mean(F_nomit_backend), np.std(F_nomit_backend)))

print('\nTensored error mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, np.mean(F_mit_backend), np.std(F_mit_backend)))

Graph-state fidelity estimates

No mitigation
F(ibmq_casablanca) = 0.729 ± 0.011

Tensored error mitigation
F(ibmq_casablanca) = 1.050 ± 0.030


## Qiskit version

In [34]:
import qiskit.tools.jupyter
%qiskit_version_table